# Earnings-Call Tone Committee — Pipeline Validation

Quick notebook to verify the end-to-end pipeline populated `db.sqlite3`
and to produce simple visualisations for a demo.

## 1. Setup

In [ ]:
import sqlite3
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams["figure.figsize"] = (10, 5)

DB_PATH = Path("../db.sqlite3")
assert DB_PATH.exists(), f"Database not found at {DB_PATH.resolve()}"
print(f"Using database: {DB_PATH.resolve()}")

## 2. Load `company_summary` into a DataFrame

In [ ]:
conn = sqlite3.connect(str(DB_PATH))

df = pd.read_sql_query(
    """
    SELECT ticker,
           call_date,
           support_ratio,
           skeptic_ratio,
           neutral_ratio,
           tone_index,
           num_questions,
           high_disagreement_ratio
    FROM   company_summary
    ORDER  BY call_date, ticker
    """,
    conn,
)

conn.close()

df["call_date"] = pd.to_datetime(df["call_date"])
df.head(10)

## 3. Basic sanity checks

In [ ]:
print(f"Rows:            {len(df)}")
print(f"Unique tickers:  {df['ticker'].nunique()}  — {sorted(df['ticker'].unique())}")
print(f"Date range:      {df['call_date'].min().date()} → {df['call_date'].max().date()}")
print(f"Total questions: {df['num_questions'].sum()}")
print()

# Filter to the latest earnings date for cross-company comparison
latest = df["call_date"].max()
df_latest = df[df["call_date"] == latest].copy().sort_values("tone_index", ascending=True)
print(f"Latest date: {latest.date()}  ({len(df_latest)} ticker(s))")
df_latest

## 4. Tone Index by Ticker (latest date)

In [ ]:
fig, ax = plt.subplots(figsize=(10, max(3, len(df_latest) * 0.6)))

# Diverging colour: green for positive tone, red for negative
colors = ["#d64541" if v < 0 else "#27ae60" for v in df_latest["tone_index"]]

ax.barh(df_latest["ticker"], df_latest["tone_index"], color=colors, edgecolor="white")
ax.axvline(0, color="grey", linewidth=0.8, linestyle="--")
ax.set_xlabel("tone_index  (support_ratio − skeptic_ratio)")
ax.set_title(f"Analyst tone_index by ticker  ({latest.date()})")

# Annotate bars with the numeric value
for i, (val, ticker) in enumerate(zip(df_latest["tone_index"], df_latest["ticker"])):
    ax.text(val + 0.01 * (1 if val >= 0 else -1), i, f"{val:+.2f}",
            va="center", ha="left" if val >= 0 else "right", fontsize=9)

plt.tight_layout()
plt.show()

## 5. Support vs Skeptic Ratios by Ticker (latest date)

In [ ]:
df_melted = df_latest.melt(
    id_vars=["ticker"],
    value_vars=["support_ratio", "skeptic_ratio"],
    var_name="metric",
    value_name="value",
)

fig, ax = plt.subplots(figsize=(10, max(3, len(df_latest) * 0.7)))

sns.barplot(
    data=df_melted,
    y="ticker",
    x="value",
    hue="metric",
    palette={"support_ratio": "#27ae60", "skeptic_ratio": "#d64541"},
    ax=ax,
)

ax.set_xlabel("Ratio")
ax.set_title(f"Support vs Skeptic ratios by ticker  ({latest.date()})")
ax.legend(title="")
plt.tight_layout()
plt.show()

## 6. Tone Index Over Time (single ticker)

In [ ]:
chosen = df["ticker"].iloc[0]
df_ticker = df[df["ticker"] == chosen].sort_values("call_date")

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(
    df_ticker["call_date"],
    df_ticker["tone_index"],
    marker="o",
    linewidth=2,
    color="#2c3e50",
)
ax.axhline(0, color="grey", linewidth=0.8, linestyle="--")
ax.fill_between(
    df_ticker["call_date"],
    df_ticker["tone_index"],
    0,
    alpha=0.15,
    color="#2c3e50",
)
ax.set_ylabel("tone_index")
ax.set_xlabel("Earnings call date")
ax.set_title(f"Tone index over time — {chosen}")

# Annotate each point
for _, row in df_ticker.iterrows():
    ax.annotate(
        f"{row['tone_index']:+.2f}",
        (row["call_date"], row["tone_index"]),
        textcoords="offset points",
        xytext=(0, 10),
        ha="center",
        fontsize=8,
    )

plt.tight_layout()
plt.show()